In [1]:
import telebot
from telebot import types
import sqlite3 as sq
import datetime
import re

bot = telebot.TeleBot("5608244416:AAGO9fj5If9Focw1VttM2vIP7yDh7XYSRhs", parse_mode=None)

# create date: 05.03.2023 and time: 12:00
date_1 = datetime.date(2023, 3, 5)
time_1 = datetime.time(12,0)

table_name = f"tour_at_{date_1.strftime('%d_%m_%Y')}_{time_1.strftime('%H_%M')}"#.replace('-', '_')
#SQL doesn`t allow '-', so i have to replace it

# create an empty dictionary to store user data
user_data = {}


@bot.message_handler(commands=['omet','start']) # comands that responded
# @bot.message_handler(content_types=['text','sticker'])
def respond(message):
    
    # open file with database
    connect = sq.connect('mus_schedule.db')
    cursor = connect.cursor()
    
    cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
            name STRING,
            quantity INTEGER,
            contact STRING UNIQUE
            );   """)
    connect.commit()
    
    text_menu=f"<b>{message.from_user.first_name}</b>.,виберіть та натисніть одну з кнопок нижче\n\
           1.Зареєструватись на найближчу екскурсію.\n\
           2.Відвідати наш сайт.\n\
           3.Залишити відгук відносно роботи електротранспорту."
    markup=types.InlineKeyboardMarkup(row_width=1) 
    button_1=types.InlineKeyboardButton(f"1.Зареєструватись на екскурсія\
      {date_1.strftime ('%d-%m-%Y')} о {time_1}",callback_data='schedule')
    button_2=types.InlineKeyboardButton("2.Відвідати офіційну сторінку КП 'OMET'",\
      url="https://oget.od.ua",callback_data='site')
    button_3=types.InlineKeyboardButton("3.Залишити відгук відносно роботи електротранспорту.",\
      callback_data='respond')
    markup.add(button_1,button_2,button_3)
    
    bot.send_message(message.chat.id,text_menu,reply_markup=markup,parse_mode='html')



@bot.callback_query_handler(func=lambda call:True)
def callback(call):
  if call.message: # check if button pushed
    """Виклик bot.register_next_step_handler(call.message, save_quantity)
    в callback функції необхідно перенести в середину умови if call.data == 'schedule'."""
    if call.data == 'schedule':
      bot.register_next_step_handler(call.message, save_quantity)
      bot.send_message(call.message.chat.id, "Напишіть будь ласка цифрами (від 1 до 50) кількість людей,\
              які планують відвідати музей КП 'ОМЕТ'\n\
              (наприклад:'12' - бот ідентифікує лише цифрову складову у цьому повідомленні)")
  



def save_quantity(message):
  quantity = message.text.strip()  # remove extra spaces from text 
  
  user_data['name'] = message.from_user.first_name
    
  # check if quantity is real number
  if not quantity.isdigit():
    bot.send_message(message.chat.id, "Невірно введена кількість")
    bot.register_next_step_handler(message, save_quantity)  # waiting for next message
    return
  
  # check if quantity more than 50
  if int(quantity) > 50:
    bot.send_message(message.chat.id, "Зв'яжіться із директором музею за телефоном 050 399 42 11 для обговорення окремої\
      екскурсії для такої великої кількості людей\n\
      Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'")
    return
  
  # separate the numerical part from the text, if there is one
  digits_str = ''.join(re.findall('\d+', quantity))
  digits_int = int(digits_str) if digits_str else None
  
  user_data['quantity'] = digits_int
  
  bot.send_message(message.chat.id, f"Кількість відвідувачів {digits_int} збережено")
  
  bot.send_message(message.chat.id, "Надішліть будь ласка ваш номер телефону для зв'язку з вами:")
  
  bot.register_next_step_handler(message, ask_phone)
                   


def ask_phone(message):
  phone_number = message.text.strip()  # remove extra spaces from text
  int_phone = int(''.join(re.findall('\d+', phone_number)))
  
  # check if phone number is valid
  #if not phone_number.isdigit() or not 6 <= len(phone_number) <= 13:
  if not re.match(r'^\+?\d{10,13}$', phone_number):
    bot.send_message(message.chat.id, "В номері невідповідна кількість цифр або він містить недопустимі символи")
    bot.register_next_step_handler(message, ask_phone)  # waiting for next message
    return
  
  user_data['phone'] = int_phone
  
  # open file with database
  connect = sq.connect('mus_schedule.db')
  cursor = connect.cursor()
    
  cursor.execute(f"Insert Or Replace Into {table_name} (name, quantity, contact) VALUES (?, ?, ?)",\
    (user_data['name'], user_data['quantity'], user_data['phone']))
  connect.commit()
    
  cursor.close()
  connect.close()

  bot.send_message(message.chat.id, f"Ваш контакт збережено. Будемо раді бачити вас у музеї, гарного дня!\
    Якщо хочете повернутись в головне меню напишіть команду '/start' або '/omet'")
  
  #bot.register_next_step_handler(message, respond)


bot.infinity_polling()


2023-03-14 15:20:13,151 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-03-14 15:20:13,153 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"


In [21]:
import re

text = "3 людей"
list_numbers = re.findall(r'\d+', text)

print(list_numbers)

text = "23 особи"

# Знаходимо всі цифри у тексті та зберігаємо їх як рядок
digits_str = ''.join(re.findall('\d+', text))

# Перетворюємо рядок з цифрами на integer
digits_int = int(digits_str)

print(digits_int)  # виведе 23

phone_number = '21456'

"""^ означає початок рядка
\+? перевіряє, чи присутній символ "+", який може бути або немає в номері телефону
\d{6,13} перевіряє, чи присутні в рядку рівно 6-13 цифр
$ означає кінець рядка"""

re.match(r'^\+?\d{6,13}$', phone_number)

['3']
23


In [10]:
import datetime

"""Зверніть увагу, що я змінив деякі частини вашого коду:
datetime.datetime.now().time() замість datetime.now().time(), щоб отримати поточний час з датою.
datetime.timedelta замість просто timedelta, оскільки ви імпортували timedelta з модуля datetime,
але не з модуля datetime.datetime.
datetime.datetime.combine(datetime.date.today(), current_time) замість datetime.combine(datetime.min,
current_time), оскільки datetime.min містить дату 1 січня 1 року, а не поточну дату, тому
datetime.datetime.combine(datetime.min, current_time) може дати неправильні результати при додаванні до
поточного часу."""

# створюємо дату 20 лютого 2023 року
date = datetime.date(2023, 2, 20)

# створюємо час 13:30:00
time = datetime.time(13, 30, 0)

# створюємо дату та час 20 лютого 2023 року о 13:30:00
datetime_obj = datetime.datetime(2023, 2, 20, 13, 30, 0)

#щоб отримати поточну дату та час
now = datetime.datetime.now()

#щоб отримати час через годину від поточного моменту
current_time = datetime.datetime.now().time()  # поточний час
delta_hours = datetime.timedelta(hours=1)  # різниця у 1 години
new_time = (datetime.datetime.combine(datetime.date.today(), current_time) + delta_hours).time() # новий час

print('new_time: ', new_time)

print('now: ',now)

print('date: ', date)
print('time: ', time)
print('datetime: ', datetime_obj)

date_1 = datetime.date(2023, 3, 5)
formatted_date = date_1.strftime('%d, %m, %Y')

print (date_1)
print(formatted_date)  # виведе '05, 03, 2023'


new_time:  17:43:44.838170
now:  2023-03-08 16:43:44.838170
date:  2023-02-20
time:  13:30:00
datetime:  2023-02-20 13:30:00
2023-03-05
05, 03, 2023
